## Trump Tweets (Pandemic!)

In [1]:
# Dependencies and Setup
import json
import os
import pandas as pd
import urllib.request
import requests
# from config import db_pwd, db_user
from sqlalchemy import create_engine

## Store Trump Tweets CSV into DataFrame

In [2]:
csv_file = "data/tweets.csv"
tweets_df = pd.read_csv(csv_file)
tweets_df.head()

,id,text,isRetweet,isDeleted,device,favorites,retweets,date
0,98454970654916608,Republicans and Democrats have both created ou...,f,f,TweetDeck,49,255,2011-08-02 18:07:48
1,1234653427789070336,I was thrilled to be back in the Great city of...,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50
2,1218010753434820614,RT @CBS_Herridge: READ: Letter to surveillance...,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47
3,1304875170860015617,The Unsolicited Mail In Ballot Scam is a major...,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58
4,1218159531554897920,RT @MZHemingway: Very friendly telling of even...,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59


In [3]:
tweets_df.columns

Index(['id', 'text', 'isRetweet', 'isDeleted', 'device', 'favorites',
       'retweets', 'date'],
      dtype='object')

In [4]:
tweets_df = tweets_df[["date", "text","favorites", "retweets"]]
tweets_df.head()

,date,text,favorites,retweets
0,2011-08-02 18:07:48,Republicans and Democrats have both created ou...,49,255
1,2020-03-03 01:34:50,I was thrilled to be back in the Great city of...,73748,17404
2,2020-01-17 03:22:47,RT @CBS_Herridge: READ: Letter to surveillance...,0,7396
3,2020-09-12 20:10:58,The Unsolicited Mail In Ballot Scam is a major...,80527,23502
4,2020-01-17 13:13:59,RT @MZHemingway: Very friendly telling of even...,0,9081


In [5]:
tweets_df.dtypes

date         object
text         object
favorites     int64
retweets      int64
dtype: object

In [6]:
# Remove the time
tweets_df['date'] = pd.to_datetime(tweets_df['date']).dt.normalize()
tweets_df

,date,text,favorites,retweets
0,2011-08-02,Republicans and Democrats have both created ou...,49,255
1,2020-03-03,I was thrilled to be back in the Great city of...,73748,17404
2,2020-01-17,RT @CBS_Herridge: READ: Letter to surveillance...,0,7396
3,2020-09-12,The Unsolicited Mail In Ballot Scam is a major...,80527,23502
4,2020-01-17,RT @MZHemingway: Very friendly telling of even...,0,9081
...,...,...,...,...
54437,2020-10-23,RT @EliseStefanik: President @realDonaldTrump ...,0,9912
54438,2020-10-23,RT @TeamTrump: LIVE: Presidential Debate #Deba...,0,8249
54439,2020-10-22,Just signed an order to support the workers of...,175950,36098
54440,2020-10-22,Suburban women want Safety &amp; Security. Joe...,95325,19639


In [7]:
# Sort the tweets by date
tweets_date = tweets_df.sort_values(['date'])
tweets_date

,date,text,favorites,retweets
11183,2009-05-04,Be sure to tune in and watch Donald Trump on L...,939,519
11182,2009-05-05,Donald Trump will be appearing on The View tom...,259,34
11180,2009-05-08,New Blog Post: Celebrity Apprentice Finale and...,29,11
11181,2009-05-08,Donald Trump reads Top Ten Financial Tips on L...,37,15
11178,2009-05-12,"""Miss USA Tara Conner will not be fired - """"I'...",25,26
...,...,...,...,...
778,2020-10-24,Presidential Promises Kept: Justice Barrett Wi...,32906,6882
779,2020-10-24,Debate Poll Average: 89% Trump. 11% Sleepy Joe...,141228,22030
491,2020-10-24,RT @marklevinshow: Another terrific Trump fore...,0,10848
493,2020-10-24,RT @GOPChairwoman: Everyone go vote early in p...,0,3214


In [8]:
# Cut the data by date - same start and end date as Google Mobility Data 2020-02-15 : 2020-08-04
start_date = '2020-02-14'
end_date = '2020-08-04'
mask = (tweets_date['date'] > start_date) & (tweets_date['date'] <= end_date)
tweet_dates = tweets_date.loc[mask]
tweet_dates

,date,text,favorites,retweets
4709,2020-02-15,RT @Mike_Pence: Under President @realDonaldTru...,0,4377
4679,2020-02-15,Thank you David! https://t.co/yElxOFxqYu,77406,15585
4708,2020-02-15,RT @GOPChairwoman: I couldn’t agree more @Lara...,0,4553
4691,2020-02-15,RT @Mike_Pence: President @realDonaldTrump pro...,0,3891
4664,2020-02-15,RT @DavidJHarrisJr: Proof it’s all about emoti...,0,29890
...,...,...,...,...
11014,2020-08-04,Whether you call it Vote by Mail or Absentee V...,91642,21145
10986,2020-08-04,RT @TeamTrump: President @realDonaldTrump: Dru...,0,9205
250,2020-08-04,RT @LouDobbs: Reopening America: President @re...,0,7241
11006,2020-08-04,RT @SecBernhardt: The Great American Outdoors ...,0,8034


In [9]:
# Query the df for specific text
covid = tweet_dates[tweet_dates['text'].str.contains("pandemic")]
covid

,date,text,favorites,retweets
3346,2020-03-13,"For decades the @CDCgov looked at, and studied...",82969,21719
3212,2020-03-13,RT @USATODAY: Viking Cruises cancels all cruis...,0,4360
4739,2020-03-17,RT @LindseyGrahamSC: President Trump’s decisio...,0,6483
5091,2020-03-18,RT @TIME: Officials said the president has aut...,0,5104
5946,2020-03-20,RT @SenHydeSmith: #Telehealth is a valuable to...,0,3237
6340,2020-03-22,RT @CVSHealth: .@cvspharmacy waiving charges f...,0,2564
7061,2020-03-22,RT @senatemajldr: It's our job to step in and ...,0,5108
7416,2020-03-23,RT @RealJamesWoods: We are in the middle of an...,0,15731
1931,2020-03-31,RT @RepMattGaetz: Our doctors and healthcare p...,0,5070
3350,2020-04-04,RT @IWF: 🤦‍♀️ @SpeakerPelosi is setting up a N...,0,4421


In [10]:
# Export the cleaned data as a csv file
tweet_dates.to_csv("data/tweet_dates.csv")

In [11]:
# Export the cleaned data as a json file
tweet_dates.to_json('data/tweet_dates.json')

### Connect to local database

In [ ]:
rds_connection_string = f"{db_user}:{{db_pwd}}@localhost:5432/mobility_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [ ]:
engine.table_names()

### Use pandas to load csv converted DataFrame into database

In [ ]:
tweet_dates.to_sql(name='trump_tweets', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the tables

In [ ]:
pd.read_sql_query('select * from trump_tweets', con=engine).head(10)